Uvozimo potrebne knjižnice za manipulacijo s podatki:

In [3]:
import random
import math

Najprej določimo točke v kvadratu velikosti 1x1. Točke so določena z dvema koordinatama (x,y), kjer x označuje premik po vodoravni stranici, y pa premik po navpični stranici. 

Oglišča kvadrata so (0,0), (0,1), (1,0), (1,1).

In [32]:
def kvadrat_tock(st_tock):
    slovar_tock = {}
    for i in range(st_tock):
        x = random.random()
        y = random.random()
        slovar_tock['tocka_' + str(i)] = (x,y)
    return slovar_tock

tocke = kvadrat_tock(8)
tocke = {'tocka_0': (0.1, 0.1), 'tocka_1': (0.2, 0.1),  'tocka_2': (0.1,
0.2),  'tocka_3': (0.2, 0.2)}

Razdalje med točkami izračunamo s pomočjo Pitagorovega izreka.

In [33]:
def razdalje(slovar):
    slovar_razdalj = {}
    #a^2 + b^2 = c^2
    for key1 in slovar.keys():
        seznam_razdalj = []
        # for key in slovar.keys()
        for key2 in slovar.keys():
            if key1 == key2:
                continue
            delta_x = slovar[key1][0] - slovar[key2][0]
            delta_y = slovar[key1][1] - slovar[key2][1]
            razdalja = sqrt(delta_x^2 + delta_y^2)
            seznam_razdalj.append((key2,razdalja))
        slovar_razdalj[key1] = seznam_razdalj
    return(slovar_razdalj)

razdalje(tocke)

{'tocka_0': [('tocka_1', 0.100000000000000),
  ('tocka_2', 0.100000000000000),
  ('tocka_3', 0.141421356237310)],
 'tocka_1': [('tocka_0', 0.100000000000000),
  ('tocka_2', 0.141421356237310),
  ('tocka_3', 0.100000000000000)],
 'tocka_2': [('tocka_0', 0.100000000000000),
  ('tocka_1', 0.141421356237310),
  ('tocka_3', 0.100000000000000)],
 'tocka_3': [('tocka_0', 0.141421356237310),
  ('tocka_1', 0.100000000000000),
  ('tocka_2', 0.100000000000000)]}

Posamezna razdalja nam predstavlja preferenco (manjša kot je razdalja, večja je preferenca). Le te uredimo po velikosti, od najmanjše do največje, za vsako točko:

In [34]:
def preference(slovar_razdalj):
    for key in slovar_razdalj.keys():
        sorted_by_second = sorted(slovar_razdalj[key], key=lambda tup: tup[1])
        slovar_razdalj[key] = sorted_by_second
    return(slovar_razdalj)

preference(razdalje(tocke))

{'tocka_0': [('tocka_1', 0.100000000000000),
  ('tocka_2', 0.100000000000000),
  ('tocka_3', 0.141421356237310)],
 'tocka_1': [('tocka_0', 0.100000000000000),
  ('tocka_3', 0.100000000000000),
  ('tocka_2', 0.141421356237310)],
 'tocka_2': [('tocka_0', 0.100000000000000),
  ('tocka_3', 0.100000000000000),
  ('tocka_1', 0.141421356237310)],
 'tocka_3': [('tocka_1', 0.100000000000000),
  ('tocka_2', 0.100000000000000),
  ('tocka_0', 0.141421356237310)]}

Algoritem je sestavljen iz dveh funkcij:
1) funkcija "Najkrajsa"  pregleda razdalje med točkami. Ko najde najmanjšo razdaljo izmed vseh, izbere tisti dve točki, med katerima ta razdalja nastopi in ju poveže v par. Na koncu nam vrne par točk, s pripadajočo najmanjšo razdaljo.

In [35]:
def Najkrajsa(slovar):
    min_razdalja = sqrt(2)
    par = []
    for key1 in slovar.keys():
        #nastavimo min_razdaljo na najmanjso ki nastopa v slovarju
        if slovar[key1][0][1] <= min_razdalja:
            min_razdalja = slovar[key1][0][1]

    for key2 in slovar.keys():
        if slovar[key2][0][1] == min_razdalja:
            par = [slovar[key2][0][0], key2]
            break

    return(par, min_razdalja)


Najkrajsa(preference(razdalje(tocke)))

(['tocka_1', 'tocka_0'], 0.100000000000000)

2) funkcija "Vsi_pari" nam omogoči, da se čez 1. funckijo zapeljemo tolikokrat kolikokrat je potrebno in povežemo vse točke v pare. Na koncu nam vrne seznam vseh parov.

In [36]:
def Vsi_pari(slovar):
    pari = []
    for i in range(len(slovar)/2):
        (par, najkrajsa) = Najkrajsa(slovar)
        for e in par:
            # Izbrišemo točki v paru iz slovarja
            if str(e) in slovar:
                del slovar[str(e)]
            # izbrišemo to preferenco še vsem točkam v slovarju
            for elm in slovar.keys():
                #print(elm, slovar[elm])
                for tocka in slovar[elm]:
                    if e in tocka:
                        slovar[elm].remove(tocka)

        pari.append(par)
    return(pari)

Vsi_pari(preference(razdalje(tocke)))

[['tocka_1', 'tocka_0'], ['tocka_3', 'tocka_2']]

S funkcijo "vsota_razdalj" izračunamo seštevek vseh razdalj med točkami, ki so skupaj v paru:

In [37]:
def vsota_razdalj(slovar, pari):
    vsota = 0
    # Pogledamo vse pare v "pari", poiščemo točko ki nastopa v slovarju in vzamemo dolžino iz njenih preferenc do točke, s katero je v paru.
    for e in pari:
        for i in range(len(slovar[e[0]])):
            if slovar[e[0]][i][0] == e[1]:
                vsota += slovar[e[0]][i][1]
    return(vsota)

vsota_razdalj(preference(razdalje(tocke)),Vsi_pari(preference(razdalje(tocke))))

0.200000000000000

Na koncu pa še s pomočjo sledeče funkcije analiziramo, kako se obnaša naša vsota razdalj, ko povečujemo število točk.

In [38]:
def razlicne_vsote(od, do, korak):
    for i in range(od, do, korak):
        yield(vsota_razdalj(preference(razdalje(kvadrat_tock(i))),Vsi_pari(preference(razdalje(kvadrat_tock(i))))))

for i in razlicne_vsote(2, 202, 10):
    print(i)

0.2618579866463865
2.422431094332822
3.87999286190593
9.642316303355795
11.14784396292636
13.603785682677044
14.622606230711824
18.32788433733497


18.798712018778595
24.48900051570604


29.077366288838604
27.753111997114967


31.2532037550905


38.09071831005373


...

39.51428777877224


39.60481025961597


48.4288551219817


52.38387520826153


49.884968582947124
